# Observed eddy kinetic energy calculations
Eddy kinetic energy will be calculated using geostrophic velocity anomalies from satellite altimetry observations.

In [ ]:
import xarray as xr
import numpy as np
from cdo import *
cdo=Cdo()

In [ ]:
vfiles=[]
ufiles=[]
for i in np.arange(1993,2020,1):
    vfiles.append('obs_vgosa_'+str(i)+'_1.nc')
    vfiles.append('obs_vgosa_'+str(i)+'_2.nc')
    vfiles.append('obs_vgosa_'+str(i)+'_3.nc')
    ufiles.append('obs_ugosa_'+str(i)+'_1.nc')
    ufiles.append('obs_ugosa_'+str(i)+'_2.nc')
    ufiles.append('obs_ugosa_'+str(i)+'_3.nc')
    
vfiles.append('obs_vgosa_'+str(2020)+'_1.nc')
vfiles.append('obs_vgosa_'+str(2020)+'_2.nc')
ufiles.append('obs_ugosa_'+str(2020)+'_1.nc')
ufiles.append('obs_ugosa_'+str(2020)+'_2.nc')

In [ ]:
vout='obs_vgosa_1993-2020.nc'
uout='obs_ugosa_1993-2020.nc'
cdo.copy(input=vfiles,output=vout)
cdo.copy(input=ufiles,output=uout)

For the highest possible level of consistency between observed calculations and EKE calculations with the modeled climate change projections, 5-day mean velocity data with February 29th removed will be used. 

In [ ]:
!cdo del29feb obs_vgosa_1993-2020_nodu.nc obs_vgosa_1993-2020_nol.nc
!cdo del29feb obs_ugosa_1993-2020_nodu.nc obs_ugosa_1993-2020_nol.nc

In [ ]:
#Create 5day means
inpu='obs_ugosa_1993-2020_nol.nc'
inpv='obs_vgosa_1993-2020_nol.nc'

outpu='obs_ugosa_1993-2020_nol_5d.nc'
outpv='obs_vgosa_1993-2020_nol_5d.nc'

cdo.timselmean(5,input=inpu,output=outpu)
cdo.timselmean(5,input=inpv,output=outpv)

Calculate EKE using cdo.

In [ ]:
#square the velocities
inpv='obs_vgosa_1993-2020_nol_5d.nc'
outpv='obs_vgosa2_1993-2020_nol_5d.nc'

cdo.mul(input=([inpv,inpv]),output=outpv)

In [ ]:
inpu='obs_ugosa_1993-2020_nol_5d.nc'
outpu='obs_ugosa2_1993-2020_nol_5d.nc'

cdo.mul(input=([inpu,inpu]),output=outpu)

In [ ]:
#add zonal and meridional velocities
inpu='obs_ugosa2_1993-2020_nol_5d.nc'
inpv='obs_vgosa2_1993-2020_nol_5d.nc'
outp='obs_v2+u2_1993-2020_nol_5d.nc'

cdo.add(input=([inpu,inpv]),output=outp)

In [ ]:
#multiply by the constant 1/2
!cdo mulc,0.5 obs_v2+u2_1993-2020_nol_5d.nc obs_eke_1993-2020_nol_5d.nc

In [ ]:
#remap the data onto the FESOM mesh for consistent comparison between datasets (i.e. to consistently remove coastal nodes and to select consistent ocean basins)
!cdo -P 8 remapbil,/path/to/mesh/data/griddes.nc obs_eke_1993-2020_nol_5d.nc obs_eke_1993-2020_nol_5d_rm.nc

In [ ]:
#apply a 1-year running mean to mask seasonality in time series analysis
inp='obs_eke_1993-2020_nol_5d_rm.nc'
outp='obs_eke_1993-2020_nol_5d_rm_runmean.nc'
cdo.runmean(73,input=inp,output=outp)

Change the metadata to reflect the cdo operations

In [ ]:
temp=xr.open_dataset('obs_eke_1993-2020_nol_5d_rm.nc')
temp=temp.rename({'ugosa':'EKE'})
temp['EKE']=temp.EKE.assign_attrs({'units':'m^2/s^2',
                                   'description':'Eddy kinetic energy computed using observed geostrophic surface velocity data from satelite altimetry. Anomalies based on 20yr reference period (1993-2012).'
                                   })

temp.to_netcdf('obs_eke_1993-2020_nol_5d_rm_mdata.nc')

In [ ]:
temp=xr.open_dataset('obs_eke_1993-2020_nol_5d_rm_runmean.nc')
temp=temp.rename({'ugosa':'EKE'})
temp['EKE']=temp.EKE.assign_attrs({'units':'m^2/s^2',
                                   'description':'Eddy kinetic energy computed using observed geostrophic surface velocity data from satelite altimetry. Anomalies based on 20yr reference period (1993-2012). 1 year running mean applied.'
                                   })

temp.to_netcdf('obs_eke_1993-2020_nol_5d_rm_runmean_mdata.nc')